In [1]:
import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 1200)
pd.set_option('display.width', 200)

---  
## Parsing a gpx file

In [23]:
track = open('data/track.gpx').read()
point_count = track.count('<trkpt')

In [24]:
point = 0
lat = []
lon = []
for i in range(point_count):
    point = track.find('<trkpt', point + 1)
    coords = track[point:track.find('\n', point)].split('"')
    lat.append(float(coords[1]))
    lon.append(float(coords[3]))

In [25]:
df_coords = pd.DataFrame({'lat':lat, 'lon':lon})
df_coords.head()

,lat,lon
0,56.30686,38.28950
1,56.30661,38.28866
2,56.30661,38.28780
3,56.30660,38.28683
4,56.30661,38.28593


In [26]:
def shift_forw(lst):
    new_lst = lst.copy()
    new_lst.insert(0, new_lst.pop())
    return new_lst
def shift_back(lst):
    new_lst = lst.copy()
    new_lst.append(new_lst.pop(0))
    return new_lst

In [27]:
df_shifted = pd.DataFrame({'prev_lat':(shift_forw(lat)-df_coords.lat), 'prev_lon':(shift_forw(lon)-df_coords.lon)})
wpt = [56.30682, 38.29001]
df_shifted['wpt_lat'] = df_coords.lat - wpt[0]
df_shifted['wpt_lon'] = df_coords.lon - wpt[1]
df_shifted.head()

,prev_lat,prev_lon,wpt_lat,wpt_lon
0,-0.00004,0.00051,0.00004,-0.00051
1,0.00025,0.00084,-0.00021,-0.00135
2,0.00000,0.00086,-0.00021,-0.00221
3,0.00001,0.00097,-0.00022,-0.00318
4,-0.00001,0.00090,-0.00021,-0.00408


---  
## Filling preferable (good) and unpreferable (bad) distances for each angle

In [2]:
polyline_good = (0x1, 0x2, 0x4, 0x6, 0x7, 0x8, 0x9, 0xa, 0xb, 0xd, 0xe, 0x16, 0x1a, \
                 0x1b, 0x1c, 0x27, 0x28, 0x29, 0x2a, 0x2b, 0x32, 0x33, 0x34, 0x35)
polyline_bad = (0x3, 0x15, 0x18, 0x19, 0x1e, 0x1f)

In [3]:
import numpy as np
import math

In [4]:
def rel_dist(x, y):
    return math.sqrt(x**2 + y**2)

In [5]:
def check_quarter(x, y, quarter_curr):
    if (quarter_curr == 1):
        return (x >= 0) and (y >= 0)
    if (quarter_curr == 2):
        return (x <= 0) and (y >= 0)
    if (quarter_curr == 3):
        return (x <= 0) and (y <= 0)
    if (quarter_curr == 4):
        return (x >= 0) and (y <= 0)

In [94]:
def vec(a, b):
    return np.array(b) - np.array(a)

def mod(v):
    return (v[0]**2 + v[1]**2)**0.5

def point_in_angle(p, a, b, c):
    delta = 0.00000001
    if (b == a) or (b == c) or (b == p):
        return 0
    return math.fabs(math.acos(np.dot(vec(b,a), vec(b,c))/(mod(vec(b, a)) * mod(vec(b,c)))) - \
        math.acos(np.dot(vec(b,a), vec(b,p))/(mod(vec(b,a)) * mod(vec(b,p)))) - \
        math.acos(np.dot(vec(b,p), vec(b,c))/(mod(vec(b,p)) * mod(vec(b,c))))) < delta

In [95]:
def fill_dist(x1, y1, x2, y2, type_line):
    x1 -= curr[0]
    y1 -= curr[1]
    x2 -= curr[0]
    y2 -= curr[1]
    if (rel_dist(x1, y1) > dist_to_wpt) and (rel_dist(x2, y2) > dist_to_wpt):
        return
    # plt.plot([y1 + curr[1], y2 + curr[1]], [x1 + curr[0], x2 + curr[0]], color='red')
    for i in range(angle_count):
        k_curr = math.tan((i * 360 / angle_count + small_decline)*math.pi/180)
        quarter_curr = math.trunc(i * 360 / angle_count / 90) + 1
        delta_angle = math.tan(math.pi/180)
        if x1 == x2:
            k_segment = 1000
        else: k_segment = (y1 - y2) / (x1 - x2)
        # if abs(k_segment - k_curr) < delta_angle:
            # go there + check quarter
        b_segment = y1 - k_segment * x1
        x = b_segment / (k_curr - k_segment)
        y = k_curr * x
        if (not check_quarter(x, y, quarter_curr)):
            continue
        if not point_in_angle((x, y), (x1, y1), (0, 0), (x2, y2)):
            continue
        dist_from_curr = rel_dist(x, y)
        if (dist_from_curr > dist_to_wpt):
            continue
        if (type_line in polyline_good) and (curr_type <= 0):
            if math.isnan(df.dist_good[i]) or (df.dist_good[i] > dist_from_curr):
                df.dist_good[i] = dist_from_curr
                df.good_x[i] = x + curr[0]
                df.good_y[i] = y + curr[1]
                # plt.scatter(df.good_y[i], df.good_x[i], color='orange');
                if (not math.isnan(df.dist_bad[i])) and (df.dist_bad[i] > df.dist_good[i]):
                    df.dist_bad[i] = np.nan
                    df.bad_x[i] = np.nan
                    df.bad_y[i] = np.nan
        if (type_line in polyline_bad) and (curr_type >= 0):
            if (math.isnan(df.dist_bad[i]) or (df.dist_bad[i] > dist_from_curr)) and (dist_from_curr < df.dist_good[i]):
                df.dist_bad[i] = dist_from_curr
                df.bad_x[i] = x + curr[0]
                df.bad_y[i] = y + curr[1]
                # plt.scatter(df.bad_y[i], df.bad_x[i], color='blue');

In [116]:
angle_count = 120
small_decline = 0.0001
wpt = [55.61205, 39.97805]
curr = [55.59582, 39.96037]
start = [55.59202, 39.95831]
curr_type = 0
wpt[0] = float("{0:.6f}".format(wpt[0] - curr[0]))
wpt[1] = float("{0:.6f}".format(wpt[1] - curr[1]))
start[0] = float("{0:.6f}".format(curr[0] - start[0]))
start[1] = float("{0:.6f}".format(curr[1] - start[1]))
dist_to_wpt = rel_dist(wpt[0], wpt[1])

In [121]:
%%time
import re

df = pd.DataFrame(index = [i for i in range(angle_count)], columns = \
                  ['dist_bad', 'bad_x', 'bad_y', 'dist_good', 'good_x', 'good_y', 'angle_wpt', 'angle_from'])

bbox = 'N55bE039'
file_mp = 'maps/' + bbox + '.mp'
slazav_map = open(file_mp, encoding = "ISO-8859-1").read()

polyline_count = slazav_map.count('[POLYLINE]')
polyline = 0
offset_to_type = 16
offset_to_coord = 7
for i in range(polyline_count):
    polyline = slazav_map.find('[POLYLINE]', polyline + 1) + offset_to_type
    type_line = int(slazav_map[polyline:slazav_map.find('\n', polyline)], 16)
    if not (type_line in polyline_good or type_line in polyline_bad):
        continue
    start_coord = slazav_map.find('Data', polyline) + offset_to_coord
    coord = re.split('\),\(|,', slazav_map[start_coord:(slazav_map.find('\n', start_coord)-1)])
    j = 2
    length = len(coord)
    while j < length:
        fill_dist(float(coord[j-2]), float(coord[j-1]), float(coord[j]), float(coord[j+1]), type_line)
        j += 2

CPU times: user 2.64 s, sys: 8.22 ms, total: 2.65 s
Wall time: 2.9 s


In [122]:
for i in range (angle_count):
    angle = abs(math.atan2(wpt[1], wpt[0]) * 180 / math.pi - i * 360 / angle_count)
    df.angle_wpt[i] = angle if angle < 180 else (360 - angle)
    angle = abs(math.atan2(start[1], start[0]) * 180 / math.pi - i * 360 / angle_count)
    df.angle_from[i] = angle if angle < 180 else (360 - angle)
df.dist_bad *= 10000 
df.dist_good *= 10000
df.head()

,dist_bad,bad_x,bad_y,dist_good,good_x,good_y,angle_wpt,angle_from
0,63.4228,55.6022,39.9604,67.5949,55.6026,39.9604,47.4485,28.4624
1,62.1468,55.602,39.9607,67.6912,55.6026,39.9607,44.4485,25.4624
2,60.9958,55.6019,39.961,67.5373,55.6025,39.9611,41.4485,22.4624
3,59.2101,55.6017,39.9613,67.2317,55.6025,39.9614,38.4485,19.4624
4,57.6797,55.6015,39.9616,67.3682,55.6024,39.9618,35.4485,16.4624


In [124]:
import matplotlib.pyplot as plt
import mplleaflet
fig = plt.figure()
plt.scatter(curr[1], curr[0], color='black')
plt.scatter(39.97805, 55.61205, color='black')
plt.plot([curr[1], 39.97805], [curr[0], 55.61205], color='black')
for i in range (angle_count):
    if not math.isnan(df.bad_y[i]):
        plt.scatter(df.bad_y[i], df.bad_x[i], color='blue')
    if not math.isnan(df.good_y[i]):
        plt.scatter(df.good_y[i], df.good_x[i], color='orange')
mplleaflet.display(fig=fig)

![](maps/map.png)